In [40]:
# !pip install python-dotenv
# !pip install openai
# !pip install -qU langchain-openai
# !pip install langchain langchain_community langchain_chroma

In [41]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI

_ = load_dotenv(find_dotenv()) # read local .env file
llm = ChatOpenAI(model="gpt-4o-mini")

In [42]:
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate


# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),)

Note: Retrievers do not always have to be linked to a vector store. It's about returning relevant chunks for a given unstructured query. There are different types of retrievers. The most common one is the vector store retriever, but we can choose from a whole raft of implementations to what suit best for the use case.


In [43]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down complex tasks into smaller, manageable steps to enhance understanding and execution. Techniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) facilitate this by prompting models to think step-by-step and explore multiple reasoning possibilities. This approach not only clarifies the task at hand but also aids in developing a structured plan for execution.'

In [44]:
# cleanup
vectorstore.delete_collection()